<a href="https://colab.research.google.com/github/enofitriyanti11/LSTM-MI3A-Kelompok-1/blob/main/LSTM_TUBES_MI3A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
df = pd.read_csv('/content/sample_data/tubes.csv')
df.head()

,text,label
0,Berantas dan usut tuntas kriminalisasi Hanfah ...,positif
1,RT Ceeek berita ini fakta kriminalisasi Hanifa...,negatif
2,RT Tolong dong masa instansi yang melindungi r...,positif
3,"Miris bgt deh. Kasian bu hanifah husein, istri...",negatif
4,RT Duh ga nyangka sih kalo ternyata Bu Hanifa...,positif


In [66]:
#import stopword
from nltk.tokenize import sent_tokenize, word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [67]:
def pre_process(text):
    # Case Folding: Lowercase
    # Merubah format teks menjadi format huruf kecil semua (lowercase).
    text = text.lower()

    # Case Folding: Removing Number
    # Menghapus karakter angka.
    text = re.sub(r"\d+", "", text)

    # Case Folding: Removing Punctuation
    # Menghapus karakter tanda baca.
    text = text.translate(str.maketrans("","",string.punctuation))

    #Case Folding: Removing Whitespace
    #Menghapus karakter kosong.
    text = text.strip()

    
    #Separating Sentences with Split () Method
    #Fungsi split() memisahkan string ke dalam list dengan spasi sebagai pemisah jika tidak ditentukan pemisahnya.
    pisah = text.split()

    #Filtering using sastrawi
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    text =  stopword.remove(text)

    return text

df['text'] = df['text'].apply(lambda x:pre_process(x))
df.head()

,text,label
0,berantas usut tuntas kriminalisasi hanfah husein,positif
1,rt ceeek berita fakta kriminalisasi hanifah hu...,negatif
2,rt dong masa instansi melindungi rakyat begini,positif
3,miris bgt deh kasian bu hanifah husein istri m...,negatif
4,rt duh ga nyangka sih kalo ternyata bu hanifa...,positif


In [68]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)
print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)

Training data size :  (560, 2)
Test data size :  (140, 2)


In [71]:
top_words = 10000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['text'])
list_tokenized_train = tokenizer.texts_to_sequences(train_df['text'])

max_review_length = 200
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = train_df['label']

In [77]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 200, 32)           320032    
                                                                 
 lstm_5 (LSTM)               (None, 100)               53200     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 373,333
Trainable params: 373,333
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(test_df['text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = test_df['label']
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)